Importing the necessary Libraries

In [1]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
import keras
from keras.utils import to_categorical, pad_sequences, Sequence
print("tf", tf.__version__, "keras", keras.__version__)



2025-08-15 19:59:39.925798: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-15 19:59:39.978324: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-15 19:59:41.201731: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


tf 2.20.0 keras 3.11.2


In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

# (任意) 初回のメモリ全確保を防ぐ
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print("Devices:", tf.config.list_physical_devices())
print("Logical GPUs:", tf.config.list_logical_devices('GPU'))

# ちょい重め計算でGPUを使うか確認
with tf.device('/GPU:0'):
    a = tf.random.normal([4000, 4000])
    b = tf.random.normal([4000, 4000])
    c = tf.matmul(a, b)
    print("matmul ok, shape:", c.shape)

# 簡単な学習（MNIST）
(x_train, y_train), _ = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_train = x_train[..., None]

model = keras.Sequential([
    layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=512, epochs=1)  # 1エポックだけ動けば十分


Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Logical GPUs: [LogicalDevice(name='/device:GPU:0', device_type='GPU')]
matmul ok, shape: (4000, 4000)
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8920 - loss: 0.4078
